In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/TIU-con/LED-Filtering-JAENS

/content/drive/MyDrive/TIU-con/LED-Filtering-JAENS


In [3]:
!pip install datasets
!pip install rouge_score
from datasets import load_dataset, load_metric
rouge = load_metric("rouge")
meteor = load_metric("meteor")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=e7fed77e5f6c8ddb82b0c7a2082ebbd49b3542b748c2295210d36bed38f2053b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


<ipython-input-3-adc500ed5e02>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)


    rouge1_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge1"]
    )["rouge1"].mid

    rouge2_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid


    rougeL_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rougeL"]
    )["rougeL"].mid

    rougeLSum_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rougeLSum"]
    )["rougeLSum"].mid


    meteor_output = meteor.compute(
        predictions=pred_str, references=label_str)['meteor']

    return {
        "rouge1_precision": round(rouge1_output.precision, 4),
        "rouge1_recall": round(rouge1_output.recall, 4),
        "rouge1_fmeasure": round(rouge1_output.fmeasure, 4),

        "rouge2_precision": round(rouge2_output.precision, 4),
        "rouge2_recall": round(rouge2_output.recall, 4),
        "rouge2_fmeasure": round(rouge2_output.fmeasure, 4),

        "rougeL_precision": round(rougeL_output.precision, 4),
        "rougeL_recall": round(rougeL_output.recall, 4),
        "rougeL_fmeasure": round(rougeL_output.fmeasure, 4),

        "rougeLsum_precision": round(rougeLsum_output.precision, 4),
        "rougeLsum_recall": round(rougeLsum_output.recall, 4),
        "rougeLum_fmeasure": round(rougeLsum_output.fmeasure, 4),

        "meteor_score": round(meteor_output, 4)
    }

In [5]:
#Checking predicted results csv files total rows.
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/TIU-con/LED-Filtering-JAENS/LED_Jaens_Filtering_result.csv")
#data

In [6]:
#Just checking a single abstract, you may skip
data_final=data[['article', 'abstract', 'predicted_abstract']]
data_final['abstract'][0]

" research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . \n previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . \n this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and neuropsychological assessment . \n group performance was compared across the following cognitive domains : simple attention / visuomotor processing speed , executive function ( e.g. , set - shifting ) , working memory , language , and memory / new verbal learning . \n results showed that pda+ performed significantly worse on the digit span forward and backward test and part b of the trail making tas

In [7]:
import csv
l=len(data_final)
with open('Rouge_score.csv', mode='w') as csv_file:
  fieldnames = ['abstract','Rouge1-F1','Rouge2-F1','RougeL-F1', 'RougeLSum-F1','METEOR_Final']
  writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
  writer.writeheader()

  #variables to calculate sum of BERT score
  sum_Rouge1_F1=0
  sum_Rouge2_F1=0
  sum_RougeL_F1=0
  sum_RougeLSum_F1=0
  sum_METEOR_F1=0
  for i in range(len(data_final)):
    Rouge1= rouge.compute(predictions=[data_final["predicted_abstract"][i]], references=[data_final["abstract"][i]], rouge_types=["rouge1"])["rouge1"].mid
    Rouge2= rouge.compute(predictions=[data_final["predicted_abstract"][i]], references=[data_final["abstract"][i]], rouge_types=["rouge2"])["rouge2"].mid
    RougeL= rouge.compute(predictions=[data_final["predicted_abstract"][i]], references=[data_final["abstract"][i]], rouge_types=["rougeL"])["rougeL"].mid
    RougeLSum= rouge.compute(predictions=[data_final["predicted_abstract"][i]], references=[data_final["abstract"][i]], rouge_types=["rougeLsum"])["rougeLsum"].mid
    METEOR=meteor.compute(predictions=[data_final["predicted_abstract"][i]], references=[data_final["abstract"][i]])['meteor']
    writer.writerow({'abstract':data_final["abstract"][i],'Rouge1-F1':round((Rouge1.fmeasure),4),'Rouge2-F1':round((Rouge2.fmeasure),4),'RougeL-F1':round((RougeL.fmeasure),4),'RougeLSum-F1':round((RougeLSum.fmeasure),4),'METEOR_Final':round(METEOR,4)})
    sum_Rouge1_F1=sum_Rouge1_F1+round((Rouge1.fmeasure),4)
    sum_Rouge2_F1=sum_Rouge2_F1+round((Rouge2.fmeasure),4)
    sum_RougeL_F1=sum_RougeL_F1+round((RougeL.fmeasure),4)
    sum_RougeLSum_F1=sum_RougeLSum_F1+round((RougeLSum.fmeasure),4)
    sum_METEOR_F1=sum_METEOR_F1+round(METEOR,4)
  Avg_Rouge1_F1=sum_Rouge1_F1/l
  Avg_Rouge2_F1=sum_Rouge2_F1/l
  Avg_RougeL_F1=sum_RougeL_F1/l
  Avg_RougeLSum_F1=sum_RougeLSum_F1/l
  Avg_METEOR_F1=sum_METEOR_F1/l
  print(round(Avg_Rouge1_F1,4))
  print(round(Avg_Rouge2_F1,4))
  print(round(Avg_RougeL_F1,4))
  print(round(Avg_RougeLSum_F1,4))
  print(round(Avg_METEOR_F1,4))

  #save in a text file
with open("ROUGE_METEOR.txt","w") as f:
  f.write("Avg_Rouge1score:" + str(Avg_Rouge1_F1) +'\n')
  f.write("Avg_Rouge2score:" + str(Avg_Rouge2_F1) +'\n')
  f.write("Avg_RougeLscore:" + str(Avg_RougeL_F1) +'\n')
  f.write("Avg_RougeLSumscore:" + str(Avg_RougeLSum_F1) +'\n')
  f.write("Avg_Meteorscore:" + str(Avg_METEOR_F1) +'\n')




0.4007
0.1403
0.2073
0.3424
0.2879
